# 将事件发送到 S3

此笔记本非常简单，只需要足够的代码就可以根据 Amazon Personalize 的用户交互将事件作为离散的 JSON 文件发送到 S3。

## 设置

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

请务必更新下面的活动和数据集 ARN。

In [ ]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Campaign ARN
Campaign_ARN = "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-demo-camp"

# Dataset Group Arn:
datasetGroupArn = "arn:aws:personalize:us-east-1:059124553121:dataset-group/personalize-lambda-demo"

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')


# S3 Configuration
bucket = "personalizedemo9000chris"

## 伪造事件

此代码将依赖于我们碰巧知道存在于 MovieLens 数据集中的 3 个用户：

1. 196
2. 13
3. 198

它们是随机选择的，我们现在还将随机选择 3 部电影供用户与之交互：

1. 225
2. 203
3. 242

这些用户将分别与每部电影进行交互，并且每个交互都将作为不同的 JSON 文件写入 S3。

In [ ]:
users = ["196", "13", "198"]
items = ["225", "203", "242"]

In [ ]:
def write_interaction_to_JSON(user, item):
    # An event needs a user, session, time, type, and a properties dictionary.
    time_of_interaction = int(time.time())
    # This is set for simplification
    session_id = user
    user_id = user
    event_type = "watched"
    properties = {
        'itemId': str(item)
    }
    properties = json.dumps(properties)
    
    event_headers = {
        'userId': user,
        'sessionId': session_id
    }
    
    event_content = {
        'sentAt': time_of_interaction,
        'eventType': event_type,
        'properties': properties
    }
    event_list = [event_content]
    json_struct = [event_headers, event_content]
    json_filename = str(time_of_interaction) + ".json"
    
    with open(json_filename, 'w') as outfile:
        json.dump(json_struct, outfile)
    
    return json_filename

def send_interactions_to_s3():
    for user in users:
        for item in items:
            filename = write_interaction_to_JSON(user, item)
            boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)
            print(filename)
            # Sleep needed to make sure everyone gets a nice new timestamp
            time.sleep(2)
            

In [ ]:
send_interactions_to_s3()

In [ ]:
# The code below is just to explore what to put in our Lambda:

with open("1568076414.json", 'r') as input_file:
    content = json.loads(input_file.read())
    # assume tracking ID was provided via an environment variable and replace
    trackingId = "a68baf70-f0f2-4c66-af2d-e11047a0372f"
    print(content[0]['userId'])
    
    personalize_events.put_events(
        trackingId = trackingId,
        userId = content[0]['userId'],
        sessionId = content[0]['sessionId'],
        eventList = [content[1]]
    )